In [32]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
import pandas as pd

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


In [2]:
model = read_sbml_model('iML1515.xml')

2nd strategy: Introduce tryptophan hydroxylase and synthesis and regeneration of its co-factor 

In [7]:
#look for metabolites in the model and genes to knock out


for gene in model.metabolites.query('hydropterin', 'name'):
    print(gene.name, gene.id)

6-hydroxymethyl-dihydropterin pyrophosphate 6hmhptpp_c
6-carboxy-5,6,7,8-tetrahydropterin cph4_c
6-hydroxymethyl dihydropterin 6hmhpt_c


1. tryptophan hydroxylase generates 5-hydroxytryptophan: oxygen + tryptophan (trp) + tetrahydrobiopterin (BH4) <-> 5-hydroxytryptophan (5-HTP) +  4a-hydroxytetrahydrobiopterin (hydro-BH4)

In [8]:
#add metabolites to the system
tetrahydrobiopterin = Metabolite(id='thbpt_c', compartment='c')
hydroxytryptophan = Metabolite(id='htrp_c', compartment='c')
four_alpha_hydroxytetrahydrobiopterin = Metabolite(id='4a_hthbpt_c', compartment='c')

In [9]:
#design reaction
tph = Reaction('tph')
tph.add_metabolites({tetrahydrobiopterin: -1,
                      model.metabolites.o2_c: -1,
                      model.metabolites.trp__L_c: -1,
                      four_alpha_hydroxytetrahydrobiopterin: 1,
                      hydroxytryptophan: 1
                     })
print(tph.build_reaction_string())

o2_c + thbpt_c + trp__L_c --> 4a_hthbpt_c + htrp_c


In [10]:
#add reaction to the system
model.add_reactions([tph])
model.reactions.tph

Reaction identifier,tph
Name,
Memory address,0x07fe99c9adfa0
Stoichiometry,o2_c + thbpt_c + trp__L_c --> 4a_hthbpt_c + htrp_c O2 O2 + + L-Tryptophan --> +
GPR,
Lower bound,0.0
Upper bound,1000.0


In [11]:
five_hydroxytryptophan_exchange = Reaction('EX_htrp')
five_hydroxytryptophan_exchange.add_metabolites({model.metabolites.htrp_c: -1})
model.add_reaction(five_hydroxytryptophan_exchange)

2. BH4 synthesis

GTP cyclohydrolase I: GTP + water <-> dihydromonapterin-triphosphate + formate + H+

In [17]:
#design reaction
gchi = Reaction('gchi')
gchi.add_metabolites({model.metabolites.gtp_c: -1,
                      model.metabolites.h2o_c: -1,
                      model.metabolites.dhmptp_c: 1,
                      model.metabolites.for_c: 1,
                      model.metabolites.h_c: 1
                     })
print(gchi.build_reaction_string())

gtp_c + h2o_c --> dhmptp_c + for_c + h_c


In [18]:
#add reaction to the system
model.add_reactions([gchi])
model.reactions.gchi

Reaction identifier,gchi
Name,
Memory address,0x07fe99ca02af0
Stoichiometry,gtp_c + h2o_c --> dhmptp_c + for_c + h_c GTP C10H12N5O14P3 + H2O H2O --> Dihydromonapterin-triphosphate + Formate + H+
GPR,
Lower bound,0.0
Upper bound,1000.0


6-pyruvate-tetrahydropterin synthase: dihydromonapterin-triphosphate <-> triphosphate + 6-pyruvoyltetrahydropterin

In [33]:
#add metabolites
six_pyruvoyltetrahydropterin = Metabolite(id='6pthp_c', compartment='c')

In [34]:
#design reaction
ptsp = Reaction('ptsp')
ptsp.add_metabolites({model.metabolites.dhmptp_c: -1,
                      model.metabolites.pppi_c: 1,
                      six_pyruvoyltetrahydropterin: 1,
                     })
print(ptsp.build_reaction_string())

dhmptp_c --> 6pthp_c + pppi_c


In [31]:
#add reaction to the system
model.add_reactions([ptsp])
model.reactions.ptsp

Ignoring reaction 'ptsp' since it already exists.


Reaction identifier,ptsp
Name,
Memory address,0x07fe9b8242550
Stoichiometry,dhmptp_c --> 6pthp_c + pppi_c Dihydromonapterin-triphosphate --> + Inorganic triphosphate
GPR,
Lower bound,0.0
Upper bound,1000.0


sepiapterin reductase: 2 H+ + 2 NADPH + 6-pyruvoyltetrahydropterin <-> 2 NADP+ + tetrahydrobiopterin (BH4)

In [36]:
#design reaction
spr = Reaction('spr')
spr.add_metabolites({model.metabolites.h_c: -2,
                      model.metabolites.nadph_c: -2,
                      six_pyruvoyltetrahydropterin: -1,
                      model.metabolites.nadp_c: 2,
                      model.metabolites.thbpt_c: 1
                     })
print(spr.build_reaction_string())

6pthp_c + 2 h_c + 2 nadph_c --> 2 nadp_c + thbpt_c


In [37]:
#add reaction to the system
model.add_reactions([spr])
model.reactions.spr

Reaction identifier,spr
Name,
Memory address,0x07fe99c968e20
Stoichiometry,6pthp_c + 2 h_c + 2 nadph_c --> 2 nadp_c + thbpt_c + 2 H+ + 2 Nicotinamide adenine dinucleotide phosphate - reduced --> 2 Nicotinamide adenine dinucleotide phosphate +
GPR,
Lower bound,0.0
Upper bound,1000.0


3. BH4 regeneration

pterin-4alpha-carbinolamine dehydratase (PCD): 4a-hydroxytetrahydrobiopterin (hydro-BH4) <-> dihydrobiopterin (BH2) + water 

In [38]:
#add metabolites
dihydrobiopterin = Metabolite(id='dhbpt_c', compartment='c')

In [39]:
#design reaction
pcd = Reaction('pcd')
pcd.add_metabolites({four_alpha_hydroxytetrahydrobiopterin: -1,
                      dihydrobiopterin: -1,
                      model.metabolites.h2o_c: 1
                     })
print(pcd.build_reaction_string())

4a_hthbpt_c + dhbpt_c --> h2o_c


In [40]:
#add reaction to the system
model.add_reactions([pcd])
model.reactions.pcd

Reaction identifier,pcd
Name,
Memory address,0x07fe99ca02760
Stoichiometry,4a_hthbpt_c + dhbpt_c --> h2o_c + --> H2O H2O
GPR,
Lower bound,0.0
Upper bound,1000.0


dihydropteridine reductase (DHPR): dihydrobiopterin (BH2) + H+ + NADPH <-> NADP+ + tetrahydrobiopterin (BH4)

In [42]:
#design reaction
dhpr = Reaction('dhpr')
dhpr.add_metabolites({model.metabolites.dhbpt_c: -1,
                      model.metabolites.h_c: -1,
                      model.metabolites.nadph_c: -1,
                      model.metabolites.nadp_c: 1,
                      model.metabolites.thbpt_c: 1
                     })
print(dhpr.build_reaction_string())

dhbpt_c + h_c + nadph_c --> nadp_c + thbpt_c


In [43]:
#add reaction to the system
model.add_reactions([dhpr])
model.reactions.dhpr

Reaction identifier,dhpr
Name,
Memory address,0x07fe99c9e8400
Stoichiometry,dhbpt_c + h_c + nadph_c --> nadp_c + thbpt_c + H+ + Nicotinamide adenine dinucleotide phosphate - reduced --> Nicotinamide adenine dinucleotide phosphate +
GPR,
Lower bound,0.0
Upper bound,1000.0


In [44]:
# check if the growth is possible with the reactions added
model.optimize()

,fluxes,reduced_costs
CYTDK2,0.00000,-7.523353e-03
XPPT,0.00000,-1.504671e-02
HXPRT,0.00000,-1.504671e-02
NDPK5,0.00000,-0.000000e+00
SHK3Dr,0.33424,1.387779e-17
...,...,...
gchi,0.00000,0.000000e+00
ptsp,0.00000,-3.178616e-01
spr,0.00000,-1.504671e-02
pcd,0.00000,0.000000e+00


In [51]:
# change objective value to production of 5-HTP, check if E.coli still grows
with model:
    model.objective = model.reactions.tph
    max_5HTP_production = model.optimize().objective_value
    print(max_5HTP_production)

0.0


In [52]:
#optimize model with knocked out gene
with model:
    model.genes.b3708.knock_out()
    model.objective = model.reactions.tph
    max_5HTP_production = model.optimize().objective_value
    print(max_5HTP_production)

0.0


In [55]:
#optimize model with glycerol as a source of carbon
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 0
    medium['EX_glyc_e'] = 10
    model.medium = medium
    model.objective = model.reactions.tph
    max_5HTP_production = model.optimize().objective_value
    print(max_5HTP_production) 

0.0
